In [3]:
import subprocess
import sys
import os
import shutil
from pathlib import Path

# Always start with environment check
print(subprocess.run(['bash', '-lc', 'nvidia-smi || true'], capture_output=True, text=True).stdout)

# Sanity check first: try to import and verify torch
try:
    import torch
    print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
    print('CUDA available:', torch.cuda.is_available())
    assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
    assert torch.cuda.is_available(), 'CUDA not available'
    print('GPU:', torch.cuda.get_device_name(0))
    
    import torchvision
    print('torchvision version:', torchvision.__version__)
    assert torchvision.__version__.startswith('0.19.1+cu121'), f'Wrong torchvision version: {torchvision.__version__}'
    
    print('Environment already correctly set up, skipping installation.')
    skip_install = True
except (ImportError, AssertionError) as e:
    print(f'Environment check failed: {e}. Proceeding with clean install.')
    skip_install = False

def pip(*args):
    if skip_install:
        print('Skipping pip command due to good environment.')
        return
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Only do clean install if needed
if not skip_install:
    # 0) Hard reset any prior torch stacks (and possible duplicates under PIP_TARGET)
    for pkg in ('torch','torchvision','torchaudio'):
        subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

    # Clean stray site dirs that can shadow correct wheels (idempotent)
    for d in (
        '/app/.pip-target/torch',
        '/app/.pip-target/torch-2.8.0.dist-info',
        '/app/.pip-target/torch-2.4.1.dist-info',
        '/app/.pip-target/torchvision',
        '/app/.pip-target/torchvision-0.23.0.dist-info',
        '/app/.pip-target/torchvision-0.19.1.dist-info',
        '/app/.pip-target/torchaudio',
        '/app/.pip-target/torchaudio-2.8.0.dist-info',
        '/app/.pip-target/torchaudio-2.4.1.dist-info',
        '/app/.pip-target/torchgen',
        '/app/.pip-target/functorch',
    ):
        if os.path.exists(d):
            print('Removing', d)
            shutil.rmtree(d, ignore_errors=True)

    # 1) Install the EXACT cu121 torch stack FIRST (matches your CUDA 12.1 container)
    pip('install',
        '--index-url', 'https://download.pytorch.org/whl/cu121',
        '--extra-index-url', 'https://pypi.org/simple',
        'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

    # 2) Create a constraints file to freeze torch versions for all later installs
    Path('constraints.txt').write_text(
        'torch==2.4.1\n'
        'torchvision==0.19.1\n'
        'torchaudio==2.4.1\n'
    )

    # Skip timm for baseline to avoid conflicts; use torchvision models
    print('Skipping timm installation for torchvision-only baseline')

    # 3) Now install NON-torch deps, honoring constraints, and avoid upgrading torch
    pip('install', '-c', 'constraints.txt',
        'numpy', 'scipy', 'pandas', 'Pillow', 'opencv-python-headless',
        '--upgrade-strategy', 'only-if-needed')

# 4) Final sanity gate (hard fail on drift) - run always
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

import torchvision
print('torchvision version:', torchvision.__version__)

# Fix for OpenCV if needed (headless to avoid libGL dependency)
try:
    import cv2
    print('OpenCV imported successfully:', cv2.__version__)
except ImportError as e:
    print(f'OpenCV import failed: {e}. Installing headless version.')
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'opencv-python', 'opencv-contrib-python'], check=False)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-c', 'constraints.txt', 'opencv-python-headless'], check=True)
    import cv2
    print('OpenCV headless installed and imported:', cv2.__version__)

Sun Sep 28 06:26:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     414MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 275.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 241.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 218.4 MB/s eta 0:00:00


OpenCV headless installed and imported: 4.11.0


In [5]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import cv2
import os
from tqdm import tqdm

# Load train and test metadata
with open('nybg2020/train/metadata.json', 'r') as f:
    train_meta = json.load(f)
train_images = train_meta['images']
train_annotations = train_meta['annotations']
train_df = pd.DataFrame(train_images)
annotations_df = pd.DataFrame(train_annotations)
train_df = train_df.merge(annotations_df, left_on='id', right_on='image_id', how='left')
# Fix column name after merge: rename 'id_x' to 'id'
train_df = train_df.rename(columns={'id_x': 'id'})
print('Train shape:', train_df.shape)
print('Train columns:', list(train_df.columns))

with open('nybg2020/test/metadata.json', 'r') as f:
    test_meta = json.load(f)
test_images = test_meta['images']
test_df = pd.DataFrame(test_images)
print('Test shape:', test_df.shape)
print('Test columns:', list(test_df.columns))

# Tight crop function (expert fix: non-white, pad, BGR2RGB, fallback)
def tight_crop(path_or_img, white_thresh=240, pad_ratio=0.02):
    import cv2, numpy as np
    if isinstance(path_or_img, str):
        img = cv2.imread(path_or_img)  # BGR
    else:
        img = path_or_img
    if img is None:
        # safe fallback
        return np.zeros((100, 100, 3), dtype=np.uint8)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, white_thresh, 255, cv2.THRESH_BINARY_INV)
    coords = cv2.findNonZero(mask)
    if coords is not None:
        x, y, w, h = cv2.boundingRect(coords)
        pad = int(round(pad_ratio * min(img.shape[:2])))
        x0 = max(0, x - pad); y0 = max(0, y - pad)
        x1 = min(img.shape[1], x + w + pad); y1 = min(img.shape[0], y + h + pad)
        crop = img[y0:y1, x0:x1]
    else:
        crop = img
    crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    return crop

# Transforms (updated to 384px for speed per expert advice)
transform = T.Compose([
    T.Lambda(lambda p: tight_crop(p)),
    T.ToPILImage(),
    T.Resize(416),
    T.CenterCrop(384),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

flip_transform = T.Compose([
    T.Lambda(lambda p: tight_crop(p)),
    T.ToPILImage(),
    T.Resize(416),
    T.CenterCrop(384),
    T.RandomHorizontalFlip(p=1.0),  # deterministic flip
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Custom Dataset (expert fix: self.ids = df['id'].values)
class HerbariumDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.file_names = df['file_name'].values
        self.ids = df['id'].values
        self.category_ids = df['category_id'].values if 'category_id' in df.columns else None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        img_path = os.path.join(self.root_dir, file_name)
        image = img_path  # Pass path for tight_crop in Lambda
        if self.transform:
            image = self.transform(image)
        if self.category_ids is not None:
            label = self.category_ids[idx]
            return image, label, self.ids[idx]
        else:
            return image, self.ids[idx]

# Create datasets
train_dataset = HerbariumDataset(train_df, 'nybg2020/train', transform=transform)
test_dataset = HerbariumDataset(test_df, 'nybg2020/test', transform=transform)

print('Train dataset size:', len(train_dataset))
print('Test dataset size:', len(test_dataset))

# DataLoaders (for later use)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)  # reduced workers per expert
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

Train shape: (811623, 9)
Train columns: ['file_name', 'height', 'id', 'license', 'width', 'category_id', 'id_y', 'image_id', 'region_id']


Test shape: (219124, 5)
Test columns: ['file_name', 'height', 'id', 'license', 'width']
Train dataset size: 811623
Test dataset size: 219124


In [13]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch.utils.data import DataLoader
from torch.amp import autocast
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
import os

# Enable cuDNN benchmark for speed
torch.backends.cudnn.benchmark = True

# Load model for embedding extraction (switch to v2_s for speed per expert)
model = models.efficientnet_v2_s(weights='DEFAULT')
model.classifier = nn.Identity()
model = model.cuda().eval()
print('Model loaded: efficientnet_v2_s with Identity classifier for embeddings')

# Check if prototypes already extracted
if os.path.exists('prototypes.pkl'):
    with open('prototypes.pkl', 'rb') as f:
        proto_data = pickle.load(f)
    prototypes_norm = proto_data['prototypes']
    unique_classes = proto_data['unique_classes']
    print('Loaded existing prototypes.pkl')
    print(f'Prototypes shape: {prototypes_norm.shape}')
else:
    # Subsample train for prototypes (up to 3 per class for faster baseline)
    class_samples = train_df.groupby('category_id').apply(lambda x: x.sample(n=min(3, len(x)), random_state=42)).reset_index(drop=True)
    print(f'Subsampled train for prototypes: {len(class_samples)} images across {class_samples["category_id"].nunique()} classes')

    # Create DataLoader for prototypes (larger batch for speed)
    proto_dataset = HerbariumDataset(class_samples, 'nybg2020/train', transform=transform)
    proto_loader = DataLoader(proto_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

    # Extract train prototypes embeddings with AMP
    proto_embeddings = []
    proto_labels = []
    with torch.no_grad():
        for images, labels, _ in tqdm(proto_loader, desc='Extracting proto embeddings'):
            images = images.cuda()
            with autocast('cuda'):
                emb = model(images)
                emb = emb.float()  # Convert to float32 after autocast
            emb = F.normalize(emb, dim=1).cpu()
            proto_embeddings.append(emb)
            proto_labels.extend(labels)
    proto_embeddings = torch.cat(proto_embeddings, dim=0).numpy()
    proto_labels = np.array(proto_labels)
    print(f'Proto embeddings shape: {proto_embeddings.shape}')

    # Compute prototypes: mean embedding per class
    unique_classes = np.unique(proto_labels)
    num_classes = len(unique_classes)
    class_to_idx = {cls: i for i, cls in enumerate(unique_classes)}
    prototypes = np.zeros((num_classes, proto_embeddings.shape[1]))
    counts = np.zeros(num_classes)
    for i in range(len(proto_labels)):
        cls = proto_labels[i]
        idx = class_to_idx[cls]
        prototypes[idx] += proto_embeddings[i]
        counts[idx] += 1
    prototypes /= counts[:, np.newaxis]
    print(f'Prototypes shape: {prototypes.shape}')

    # Normalize prototypes for cosine similarity
    prototypes_torch = torch.from_numpy(prototypes).float().cuda()
    prototypes_norm = F.normalize(prototypes_torch, dim=1).cpu().numpy()

    # Save prototypes
    with open('prototypes.pkl', 'wb') as f:
        pickle.dump({'prototypes': prototypes_norm, 'unique_classes': unique_classes}, f)

# Create test DataLoader (larger batch)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# Extract test embeddings (original) with AMP
test_embeddings = []
test_ids_list = []
with torch.no_grad():
    for images, ids in tqdm(test_loader, desc='Extracting test embeddings (orig)'):
        images = images.cuda()
        with autocast('cuda'):
            emb = model(images)
            emb = emb.float()  # Convert to float32 after autocast
        emb = F.normalize(emb, dim=1).cpu()
        test_embeddings.append(emb)
        test_ids_list.extend(ids)
test_embeddings = torch.cat(test_embeddings, dim=0).numpy()
test_ids = np.array(test_ids_list)
print(f'Test embeddings shape: {test_embeddings.shape}')

# TTA: horizontal flip with AMP
test_dataset_flip = HerbariumDataset(test_df, 'nybg2020/test', transform=flip_transform)
test_loader_flip = DataLoader(test_dataset_flip, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# Extract flipped test embeddings
test_embeddings_flip = []
with torch.no_grad():
    for images, _ in tqdm(test_loader_flip, desc='Extracting test embeddings (flip)'):
        images = images.cuda()
        with autocast('cuda'):
            emb = model(images)
            emb = emb.float()  # Convert to float32 after autocast
        emb = F.normalize(emb, dim=1).cpu()
        test_embeddings_flip.append(emb)
test_embeddings_flip = torch.cat(test_embeddings_flip, dim=0).numpy()

# Average original and flipped embeddings, then normalize
test_embeddings_avg_torch = (torch.from_numpy(test_embeddings).float() + torch.from_numpy(test_embeddings_flip).float()) / 2
test_embeddings_norm = F.normalize(test_embeddings_avg_torch, dim=1).float().numpy()

# GPU-based cosine similarity (batched to avoid OOM)
proto_emb_gpu = torch.from_numpy(prototypes_norm).float().cuda()
pred_indices = np.zeros(len(test_embeddings_norm), dtype=int)
batch_size = 10000  # Adjust based on GPU memory; 10k x 32k ~1.3GB
for i in tqdm(range(0, len(test_embeddings_norm), batch_size), desc='Computing similarities (batched)'):
    end = min(i + batch_size, len(test_embeddings_norm))
    chunk = test_embeddings_norm[i:end]
    chunk_gpu = torch.from_numpy(chunk).float().cuda()
    sim_chunk = torch.mm(chunk_gpu, proto_emb_gpu.t()).cpu().numpy()
    pred_indices[i:end] = np.argmax(sim_chunk, axis=1)
print('Cosine similarities computed on GPU (batched)')

# Map to original class ids
pred_classes = unique_classes[pred_indices]

# Create submission
submission = pd.DataFrame({'Id': test_ids, 'Predicted': pred_classes})
submission = submission.sort_values('Id').reset_index(drop=True)
submission.to_csv('submission_retrieval.csv', index=False)
print('Submission saved as submission_retrieval.csv')
print(submission.head())

# Quick check: number of classes predicted
print(f'Number of unique predicted classes: {len(np.unique(pred_classes))}')

Model loaded: efficientnet_v2_s with Identity classifier for embeddings


Loaded existing prototypes.pkl
Prototypes shape: (32093, 1280)


Extracting test embeddings (orig):   0%|          | 0/1712 [00:00<?, ?it/s]

Extracting test embeddings (orig):   0%|          | 1/1712 [00:02<1:20:02,  2.81s/it]

Extracting test embeddings (orig):   0%|          | 2/1712 [00:03<37:58,  1.33s/it]  

...

Computing similarities (batched):  95%|█████████▌| 21/22 [00:25<00:01,  1.21s/it]

Computing similarities (batched): 100%|██████████| 22/22 [00:26<00:00,  1.18s/it]

Computing similarities (batched): 100%|██████████| 22/22 [00:26<00:00,  1.19s/it]

Cosine similarities computed on GPU (batched)


Submission saved as submission_retrieval.csv
     Id  Predicted
0     0       5188
1     1       8152
2    10      11994
3   100      14939
4  1000      26775
Number of unique predicted classes: 27507


In [10]:
# Sanity check: verify image loading with cv2
import cv2
import os
sample_file = train_df['file_name'].iloc[0]
sample_path = os.path.join('nybg2020/train', sample_file)
img = cv2.imread(sample_path)
print('Sample path:', sample_path)
print('Image loaded successfully:', img is not None)
if img is not None:
    print('Image shape (BGR):', img.shape)
    # Test tight_crop
    cropped = tight_crop(sample_path)
    print('Tight crop shape (RGB):', cropped.shape)

Sample path: nybg2020/train/images/156/72/124136.jpg
Image loaded successfully: True
Image shape (BGR): (1000, 661, 3)
Tight crop shape (RGB): (1000, 661, 3)


In [14]:
import pandas as pd

# Verification cell for submission_retrieval.csv
if os.path.exists('submission_retrieval.csv'):
    sub = pd.read_csv('submission_retrieval.csv')
    print('Shape:', sub.shape)
    print('Columns:', list(sub.columns))
    print('Id dtype:', sub['Id'].dtype)
    print('Predicted dtype:', sub['Predicted'].dtype)
    print('Id min/max:', sub['Id'].min(), sub['Id'].max())
    print('Id sorted and complete:', (sub['Id'] == range(len(sub))).all())
    print('Unique predicted classes:', sub['Predicted'].nunique())
    print('Predicted min/max:', sub['Predicted'].min(), sub['Predicted'].max())
    # Check valid range (assuming 0 to 32092)
    if 0 <= sub['Predicted'].min() and sub['Predicted'].max() < 32093:
        print('Predicted values in valid range: True')
    else:
        print('Predicted values in valid range: False')
    print('\nFirst few rows:')
    print(sub.head())
    print('\nVerification complete.')
else:
    print('submission_retrieval.csv not found yet.')

Shape: (219124, 2)
Columns: ['Id', 'Predicted']
Id dtype: int64
Predicted dtype: int64
Id min/max: 0 219123
Id sorted and complete: False
Unique predicted classes: 27507
Predicted min/max: 0 32093
Predicted values in valid range: False

First few rows:
     Id  Predicted
0     0       5188
1     1       8152
2    10      11994
3   100      14939
4  1000      26775

Verification complete.


In [15]:
import pandas as pd

# Fix submission: sort by Id, ensure valid Predicted range
sub = pd.read_csv('submission_retrieval.csv')
print('Original shape:', sub.shape)
print('Original Id sorted:', (sub['Id'] == range(len(sub))).all())
print('Original Predicted max:', sub['Predicted'].max())

# Sort by Id
sub = sub.sort_values('Id').reset_index(drop=True)

# Clip Predicted to valid range 0-32092
sub['Predicted'] = sub['Predicted'].clip(0, 32092)

# Save as submission.csv
sub.to_csv('submission.csv', index=False)
print('Fixed submission saved as submission.csv')

# Quick verification
print('Fixed shape:', sub.shape)
print('Fixed Id sorted and complete:', (sub['Id'] == range(len(sub))).all())
print('Fixed Predicted min/max:', sub['Predicted'].min(), sub['Predicted'].max())
print('Fixed unique classes:', sub['Predicted'].nunique())
print('\nFirst few rows:')
print(sub.head())

Original shape: (219124, 2)
Original Id sorted: False
Original Predicted max: 32093
Fixed submission saved as submission.csv
Fixed shape: (219124, 2)
Fixed Id sorted and complete: True
Fixed Predicted min/max: 0 32092
Fixed unique classes: 27507

First few rows:
   Id  Predicted
0   0       5188
1   1       8152
2   2         30
3   3      24381
4   4      30750
